In [1]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import MinMaxScaler #scikit-learn==0.21.3
from xgboost import XGBClassifier #xgboost==0.90
from sklearn.manifold import TSNE
import xgboost
from sklearn.preprocessing import LabelEncoder

/opt/anaconda3/envs/trophic-mode-working-test/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/opt/anaconda3/envs/trophic-mode-working-test/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/relea

In [2]:
xgboost.__version__

'0.90'

## G1 predictions

In [3]:
data = pd.read_csv('../Field_training_data_justMMETSP.csv')
labels = pd.read_csv('../Field_training_labels_justMMETSP.csv')
data = data.reset_index(drop=True)
labels = labels.reset_index(drop=True)

idx = labels.index[labels['Trophic mode'] == 'Un']
train_labels = labels.drop(idx)
train_data = data.drop(idx)

features = pd.read_csv('../Extracted_Pfams.csv')

G1_TPM_merged_lats = pd.read_csv('../../g1Surface/G1_surface_allSamples_processed_updatedMarferret_marmicroDb2023_noOutliers_tpm_noNAPfam_fall2023_noNATaxa.csv')

In [4]:
G1_TPM_merged_lats.columns[(G1_TPM_merged_lats == 0).all()]

Index([], dtype='object')

In [5]:
features = features['Pfam'].str.split('.').str[0]

In [6]:
len(features)

1046

In [7]:
original_features = set(G1_TPM_merged_lats.columns)

In [8]:
missing_features = set(features) - original_features
missing_features

{'PF03986', 'PF04413', 'PF08017', 'PF08217', 'PF08802', 'PF09631', 'PF13427'}

In [9]:
#Add the missing features to G1_TPM_merged_lats, filled with 0 values
for feature in missing_features:
    G1_TPM_merged_lats[feature] = 0

In [10]:
# Initialize empty DataFrames to store results
excluded_pfams = pd.DataFrame()

output_predictions = pd.DataFrame()

In [11]:
for i in range (0, 30):
    # Assuming G1_TPM_merged_lats is your DataFrame
    df = G1_TPM_merged_lats.copy()

    # Calculate 10% of the columns
    num_cols = df.shape[1]
    num_cols_to_zero = int(num_cols * 0.1)

    # Randomly select 10% of the columns
    columns_to_zero = np.random.choice(df.columns, size=num_cols_to_zero, replace=False)

    # Replace the selected columns with all zero values
    df[columns_to_zero] = 0

    # df now has 10% of the columns replaced with zeroes

    excluded_pfams[f'col{i}'] = columns_to_zero

    df = df[features]

    le = LabelEncoder()

    train_data = train_data[features]

    X,y = train_data, le.fit_transform(train_labels['Trophic mode'])
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)

    model = XGBClassifier(gamma=0.0, learning_rate=0.5, max_depth=3, n_estimators=10, reg_lambda=0.)
    model.fit(X,y)

    df = scaler.transform(df)
    xg_predictions = model.predict(df)
    output_predictions[f'col{i}'] = xg_predictions

In [12]:
output_predictions.to_csv('/Users/elainathomas/Dropbox/grad/research/g1Surface/bootstrapPredictions_lambert.csv')

In [13]:
excluded_pfams.to_csv('/Users/elainathomas/Dropbox/grad/research/g1Surface/bootstrapPfamsExcluded_lambert.csv')